In [18]:
from classiq import *
import numpy as np
import math

In [23]:
# Assuming these won't change
x0 = np.array([1, 1])
M = np.array([[0, 1], [-1, 0]])

# Free to change variables
t = 1
K = 10 # Accuracy of solution - subject to change
T = math.ceil(math.log2(K+1))


In [76]:
def C(m):
   return 2 ** ((m + 1) / 2) * t ** m / math.factorial(m)

N = math.sqrt(sum([C(i) for i in range(K+1)]))

state = []

for i in range(2 ** T):
   if len(state) < K + 1:
      state.append(math.sqrt(C(i)) / N)

   else:
      state.append(0)

for n in state:
   print(n ** 2)

0.24311681029618354
0.3438190903617498
0.24311681029618354
0.11460636345391655
0.04051946838269726
0.011460636345391654
0.002701297892179818
0.0005457445878757933
9.647492472070776e-05
1.5159571885438699e-05
2.1438872160157283e-06
0
0
0
0
0


In [79]:
# TODO Initialize ancilla registers with the correct quantum states
# NOTE Our b vector is zero, therefore C-V_S2 and C-U_b do not need to be implemented, but could be for future purposes


print("Register sizes:", 1, T, 1)

@qfunc
def encode_stage(ancilla_reg_1: QBit, ancilla_reg_2: QNum, work_reg: QBit):

   def C(m):
      return 2 ** ((m + 1) / 2) * t ** m / math.factorial(m)

   state = []

   for i in range(2 ** T):
      if len(state) < K + 1:
         state.append(math.sqrt(C(i)))

      else:
         state.append(0)

   state = np.array(state) / np.linalg.norm(state)
      
   Z(ancilla_reg_1)
   inplace_prepare_amplitudes(amplitudes=state.tolist(), bound=0.001, target=ancilla_reg_2)
   H(work_reg)


@qfunc
def create_entanglement(ancilla_reg_2: QNum, work_reg: QBit):
   repeat(
      count=K + 1,
      iteration=lambda i: if_(
         condition=(i % 2 != 0),
         then=lambda: control(
               ctrl=(ancilla_reg_2 == i), stmt_block=lambda: Y(work_reg)
            ),
         ),
      )

@qfunc
def decode():
   pass

@qfunc
def main(ancilla_reg_1: Output[QBit], ancilla_reg_2: Output[QNum], work_reg: Output[QBit]):
   allocate(1, ancilla_reg_1)
   allocate(T, ancilla_reg_2)
   allocate(1, work_reg) # 2x2 matrix, will change for higher order ODEs if we get to it

   encode_stage(ancilla_reg_1, ancilla_reg_2, work_reg)
   create_entanglement(ancilla_reg_2, work_reg)

Register sizes: 1 4 1


In [80]:
# View circuit on Classiq IDE
qmod = create_model(main)
qprog = synthesize(qmod)
show(qprog)

AttributeError: 'numpy.ndarray' object has no attribute 'append'

In [73]:
a = [1, 2]

np.linalg.norm(a)

2.23606797749979